In [1]:
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from time import time

Using TensorFlow backend.


In [2]:
start_time_0 = time()

### Load file

In [3]:
lyrics = open('BSB.txt').read()
lyrics = lyrics.lower()

In [4]:
len(lyrics)

36518

### Convert char to num

In [5]:
chars = [i for i in sorted(set(lyrics))]

In [6]:
char_num = dict((char, num) for num, char in enumerate(chars))
num_char = dict((num, char) for num, char in enumerate(chars))

In [7]:
lyrics_num = [char_num[i] for i in lyrics]


In [8]:
alphabet = len(char_num)

### create sequence

In [9]:
def chop_to_sequence(seq, lyrics_num):
    """
    chop lyrics_num into segments with length seq
    return list of segments
    """
    lyrics_arr = np.asarray(lyrics_num)
    lyrics_arr = lyrics_arr / float(alphabet)
    segments = []
    next_char = []
    for i in range(0, len(lyrics_num)-seq):
        segment = lyrics_arr[i:i+seq]
        segments.append(segment)
        next_char.append(lyrics_num[i+seq])

    print("segment length:", seq)
    print('number of segments:', len(segments))
    print("chars in lyrics:", len(lyrics))
    print("")
    
    segments = np.reshape(segments, (len(segments),seq,1))
    next_char = np_utils.to_categorical(next_char)
    
    return segments, next_char
    

In [10]:
def print_time(start_time):
    print((time()-start_time)/60)

In [11]:
start_time = time()
X_all, y_all = chop_to_sequence(10, lyrics_num)
print_time(start_time)


segment length: 10
number of segments: 36508
chars in lyrics: 36518

0.001930367946624756


### LSTM model

In [12]:
def predict_next_n(n):
    """
    predict next n char from random seed
    """
    start = np.random.randint(0, len(X))
    seed = lyrics[start:start+X.shape[1]]
    pattern = X[start]
    
    chars=[]
    for i in range(n):
        pred_num = model.predict_classes(np.reshape(pattern,(1,pattern.shape[0],1)), verbose=False)
        #pred_num = np.argmax(pred_arr)
        pred_char = num_char[pred_num[0]]
        
        chars.append(pred_char)
        pattern = np.append(pattern, pred_num/float(alphabet))
        pattern = pattern[1:]
        #print(pred_arr, pred_num, pred_char, pattern)
    print("Seed:", seed)
    print("Generated:", "".join(chars))
    

### All data

In [13]:
X=X_all[:1000]
y=y_all[:1000]

In [14]:
model = Sequential()
model.add(LSTM(alphabet, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(alphabet))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 39)            6396      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 39)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 39)                12324     
_________________________________________________________________
dropout_2 (Dropout)          (None, 39)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 39)                1560      
Total params: 20,280.0
Trainable params: 20,280.0
Non-trainable params: 0.0
_________________________________________________________________


In [16]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


In [17]:
from keras.optimizers import Adam

In [18]:
adam_lr = Adam(lr=0.01)

In [19]:
model.compile(loss='categorical_crossentropy', optimizer=adam_lr, metrics=['accuracy'], )
start_time = time()
model.fit(X, y, epochs=10, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 2s - loss: 3.3544 - acc: 0.1025 - val_loss: 3.0544 - val_acc: 0.1600

In [20]:
model.compile(loss='categorical_crossentropy', optimizer=adam_lr, metrics=['accuracy'], )
start_time = time()
model.fit(X, y, epochs=600, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/600
800/800 [==============================] - 2s - loss: 3.0255 - acc: 0.1625 - val_loss: 3.0099 - val_acc: 0.1600

In [23]:
model.fit(X, y, epochs=20, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/20
800/800 [==============================] - 1s - loss: 0.4609 - acc: 0.8337 - val_loss: 0.1401 - val_acc: 0.9350

In [24]:
model.fit(X, y, epochs=1, batch_size=64, callbacks= callbacks_list, verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 1s - loss: 0.4910 - acc: 0.8362 - val_loss: 0.1662 - val_acc: 0.9150


In [25]:
model.fit(X, y, epochs=300, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/300
800/800 [==============================] - 1s - loss: 0.5410 - acc: 0.8200 - val_loss: 0.1310 - val_acc: 0.9500

In [30]:
model.fit(X, y, epochs=1, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 1s - loss: 0.4862 - acc: 0.8300 - val_loss: 0.1343 - val_acc: 0.9300

In [31]:
predict_next_n(100)

Seed: erybody
ye
Generated: ah
everybody swing
yeah
everybody swing
yeah
everybody swing
yeah
everybody swing
yeah
everybody swi


In [32]:
model.fit(X, y, epochs=300, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/300
800/800 [==============================] - 1s - loss: 0.5078 - acc: 0.8250 - val_loss: 0.1257 - val_acc: 0.9450

In [34]:
model.fit(X, y, epochs=1, batch_size=64, 
          callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 1s - loss: 0.4293 - acc: 0.8438 - val_loss: 0.1166 - val_acc: 0.9400


In [ ]:
print_time(start_time)

In [37]:
predict_next_n(100)

Seed:  you baby)
Generated: 
i wanna be with you

and i'll be waiting
until you face the truth
whan
t wisn you 
and i'll be wait


In [54]:
predict_next_n(1000)

Seed: ght is fading
you know what i 
Generated: wanna do

i wanna be with you
it's crazy but it's true
(you know it's true)
and everything i do
i wanna be with you

i'd like to know your policy
ihl i monn  ndwit  you 
thee be eeee
yo tou inye iou iewe ever to sei leena you
you yeu saue you it's erun you it's taaere
yo 
thei the thanh
in
so i segna yoiiio ihai i w iamen th whet t t di watr you
know what's wauh 
you 
and tre thu iths 
n's wanna you

ane whe true

nder toe than shin s saueueatg ind i iiliang yoath toat's mamenn that i can kie to whe te t's erany you
yntn i's lanna be with you

and w'wr yo
you

and i'likenbgyhwhaadi i'lign
iosr toen s me
ief seeth yoat saat tre you ieee
youe se i w era be wiaaa yo t's laeth you 
acd w waat  tha er wa 
ah fne yeu bnoa the tiue (ad  l'wiee oo fn wi t eeery but tt what s tageues yaai aail foe to she that'seseneos
s d lig nn th whth you ith irar so ses femza yea co i wlinng wou fo i wanna be with you

and i'll be waiting
until you face the tru

In [38]:
### Human-input seed:

def predict_next(seed, n):
    """
    predict next n char from human input seed
    """
    #seed = lyrics[start:start+X.shape[1]]
    seed = seed.lower()
    seed_num = [char_num[i] for i in seed]
    seq = X[0].shape[0]
    
    if len(seed) < seq:
        print("Error: ")
    else:
        seed_arr = chop_to_sequence(seq=seq, lyrics_num=seed_num)
        pattern = seed_arr[0][0]
        
        chars=[]
        for i in range(n):
            pred_num = model.predict_classes(np.reshape(pattern,(1,pattern.shape[0],1)), verbose=False)
            #pred_num = np.argmax(pred_arr)
            pred_char = num_char[pred_num[0]]

            chars.append(pred_char)
            pattern = np.append(pattern, pred_num/float(alphabet))
            pattern = pattern[1:]
            #print(pred_arr, pred_num, pred_char, pattern)
        print("Seed:", seed)
        print("Generated:", "".join(chars))


In [39]:
seed = """iWe've got it goin' on for years
Jam on 'cause Backstreets' got it
Come on now, everybody 
We've got it goin' on for years """


In [42]:
predict_next(seed, 30)

segment length: 10
number of segments: 113
chars in lyrics: 36518

Seed: iwe've got it goin' on for years
jam on 'cause backstreets' got it
come on now, everybody 
we've got it goin' on for years 
Generated: e
wou bace the truth
whan
t wi


In [43]:
seed = """
Am I original? (Yeah)
Am I the only one? (Yeah)
Am I sexual? (Yeah)
Am I everything you need?
You'd better rock your body now
"""

In [45]:
predict_next(seed, 100)

segment length: 10
number of segments: 117
chars in lyrics: 36518

Seed: 
am i original? (yeah)
am i the only one? (yeah)
am i sexual? (yeah)
am i everything you need?
you'd better rock your body now

Generated:  bnd sht me free
so that i wanna be with you

and i'll be waiting
until you face the truth
whan
t wi
